In [ ]:
#1 Importing necessary libraries
from openai import OpenAI
from parser import code_to_file, extract_filename, run_code

In [ ]:
#2 Setting up the API
client = OpenAI(
base_url = 'http://localhost:11434/v1',
api_key='ollama', # required, but unused
)

In [ ]:
#3 Setting up the query
query = """Write a python program to check if a given number is Fibonacci number. Input:  8, 34, 41
"""
prompt = query + """
    Format the response in the following format:
    1. plan
    2. filename
    3. Code
    
    Do not show any comments. The code should not have multi line comments. Only single line comments are permissible.
    """

In [ ]:
def code_gen_exec(query, docker_image):
    #4 Setting up the response use OpenAI's chat completion endpoint
    response = client.chat.completions.create(
    model="qwen2.5:14b-instruct-q5_K_M",
    messages=[
        {"role": "system", "content": "You are a helpful coding assistant"},
        {"role": "user", "content": prompt}
      ]
    )
    
    raw_response = response.choices[0].message.content
    
    response_refine = client.chat.completions.create(
    model="qwen2.5:14b-instruct-q5_K_M",
    messages=[
        {"role": "system", "content": "You are a helpful assistant. There are five conditions. Think carefully and follow the conditions strictly."},
        {"role": "user", "content": """You will analyze the text given, follow the instructions and provide the response. 
        Instructions:
        1. Enclose your response in following xml format tags: <code> and </code>. 
        2. Save the filename and code in dictionary format. keys will be 'filename' and 'code'.  {filename_key: filename_value, code_key: code_value}
        3. Within the dictionary, in the code key if you detect the python code use markdown blocks ```python in the beginning and ``` in the end. 
        4. Use escaped triple quotes (`\"\"\"`) for the value of the key instead of triple quotes.
        5. If you detect multi line comments in triple quotes within the value of the code, remove those comments. Use single line comments
        6. Ensure you have followed the correct openings and closings like <code> </code> tags and { } curly brackets
    
        Eample: 
        <code>
        {
          "filename": "hello.py",
          "code": ```python
          print("Hello world!")
          ```
        }
        </code>

    Here is the text for you: """ + raw_response}
          ]
        )
    
    x=response_refine.choices[0].message.content
    
    code_to_file(x)
    
    file_name = extract_filename(x)
    
    code_outcome=run_code(file_name,'septimus')
    
    return file_name, code_outcome[1]
    
    

In [ ]:
code_gen_exec(query, 'septimus')

In [ ]:
result_code_gen_exec = code_gen_exec(query, 'septimus')

In [ ]:
print(result_code_gen_exec[0])

### Next steps:

1. create a function to convert the file_name.py into the file_name_error.txt or  file_name_output.txt
2. For the code fix function, the input parameters are: query, code, error, docker image name
3. Prompting: Need to analyze the best way. maybe ask something like diagnose, identify potential cause, etc in the prompting.
4. it should overwrite the existing file and not create new file.
5. it should return status code